In [ ]:
#from google.colab import drive 
#from google.colab import auth
#auth.authenticate_user()
#drive.mount('/drive')

# New Section

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
data_dir ='/content/'
import numpy as np
class Particle(object):
    def __init__(self, name, file, mass, nlev):
        """Constructor"""
        self.file = file
        self.name = name
        self.data = np.loadtxt(f"{data_dir+file}")
        self.mass = mass*1.66054e-27
        self.nlev = nlev
    def calculate(self, temp):
        k = 1.3807e-23                                                              # Boltzmann constant
        [eps_c, g_c_exp, g_c_eps_exp, g_c_eps_exp_2] = self.calculate_en(temp)      # eps_c & SUMS
        temp = float(temp)
        Z_c = g_c_exp                                                               # Statistical Sum
        E_int = g_c_eps_exp / self.mass / Z_c                                       # E
        cv_int = k * (g_c_eps_exp_2/k/k/temp/temp/Z_c - \
                (g_c_eps_exp/k/temp/Z_c)*(g_c_eps_exp/k/temp/Z_c))/self.mass        # cV, internal
        cv_tr = 3.0 * k / 2.0 / self.mass                                           # cV, translational
        cv = cv_int + cv_tr
        print(f"Calculating for {self.name}")
        print(f"Statistical Sum Z_int,c = {Z_c}")
        print(f"eps_c = {eps_c}")
        print(f"E_int,c = {E_int}")
        print(f"cV,int = {cv_int}")
        print(f"cV,tr = {cv_tr}")
        print(f"cV = cV,int + cV,tr = {cv}\n")

class Molecule(Particle):
    def calculate_en(self, T):
        nlev = self.nlev                                                        # Number of e levels
        h = 6.6261e-34                                                          # Planck constant
        c = 299792458.0                                                         # Speed of light
        k = 1.3807e-23                                                          # Boltzmann constant
        g_i = 1
        # e_vibr = np.zeros((nlev, 1))                                          # vibrational energy
        # e_rot = np.zeros((nlev, 1, 1))                                        # rotational energy
        e_el = self.data[:, 0] * 100 * h * c                                    # electron energy from data
        e_diss = self.data[:, 7] * 100 * h * c                                  # dissociation energy from data
        b_e = self.data[:, 8] * 100                                             # B_e from data
        d_e = self.data[:, 13] * 1e-4                                           # D_e from data
        # b_i = np.zeros((nlev, 1))
        # d_i = np.zeros((nlev, 1))
        alpha_e = self.data[:, 9]                                               # alpha from data
        beta_e = self.data[:, 14]                                               # beta from data
        g_n = self.data[:, 1]                                                   # Statistical weight of the nth level
        omega = self.data[:, 2:7] * 100
        eps_c = 0.0
        g_c_exp = 0.0
        g_c_eps_exp = 0.0
        g_c_eps_exp_2 = 0.0
        # Main loop
        for n in range(0, nlev):
            i = 0
            eps_c += e_el[n]
            e_vibr_ni = 0.0
            while e_el[n] + e_vibr_ni < e_diss[n]:
                b_i_ni = b_e[n] - alpha_e[n]*(i + 0.5)
                d_i_ni = d_e[n] - beta_e[n]*(i + 0.5)
                e_vibr_ni = omega[n, 0]*(i + 0.5) - \
                            omega[n, 1] * (i + 0.5) * (i + 0.5) + \
                            omega[n, 2] * (i + 0.5) * (i + 0.5) * (i + 0.5) - \
                            omega[n, 3] * (i + 0.5) * (i + 0.5) * (i + 0.5) * (i + 0.5) + \
                            omega[n, 4] * (i + 0.5) * (i + 0.5) * (i + 0.5) * (i + 0.5) * (i + 0.5)
                e_vibr_ni *= h * c
                eps_c += e_vibr_ni
                i = i + 1
                j = 0
                e_rot_nij = 0
                while e_el[n] + e_vibr_ni + e_rot_nij < e_diss[n]:
                    g_j = 2*j + 1
                    e_rot_nij = b_i_ni * j * (j + 1) - d_i_ni * j * j * (j + 1) * (j + 1)
                    e_rot_nij *= h * c
                    eps_c += e_rot_nij
                    g_c_exp += g_n[n]*g_i*g_j*np.exp(-eps_c/k/T)
                    g_c_eps_exp += g_n[n]*g_i*g_j*np.exp(-eps_c/k/T)*eps_c
                    g_c_eps_exp_2 += g_n[n]*g_i*g_j*np.exp(-eps_c/k/T)*eps_c*eps_c
                    j = j + 1
        return eps_c, g_c_exp, g_c_eps_exp, g_c_eps_exp_2


class Atom(Particle):
  def calculate_en(self, T):
    nlev = self.nlev                                                            # Number of e levels
    h = 6.6261e-34                                                              # Planck constant
    c = 299792458.0                                                             # Speed of light
    k = 1.3807e-23                                                              # Boltzmann constant
    g_n = self.data[:, 0]                                                       # Statistical weigth of the nth level                                                   
    e_el = self.data[:, 1] * 100 * h * c
    eps_c = 0.0
    g_c_exp = 0.0
    g_c_eps_exp = 0.0
    g_c_eps_exp_2 = 0.0
    # Main loop
    for n in range(0, nlev):
        eps_c += e_el[n]
        g_c_exp += g_n[n]*np.exp(-eps_c/k/T)
        g_c_eps_exp += g_n[n]*np.exp(-eps_c/k/T)*eps_c
        g_c_eps_exp_2 += g_n[n]*np.exp(-eps_c/k/T)*eps_c*eps_c
    return eps_c, g_c_exp, g_c_eps_exp, g_c_eps_exp_2

if __name__ == "__main__":
    oxygen = Molecule('Oxygen, O2', 'oxygen.txt', 15.999*2, 5)
    oxygen.calculate(1000)
    carbon = Atom('Carbon, C', 'carbon.txt', 12.0107, 19)
    carbon.calculate(1000)

Calculating for Oxygen, O2
Statistical Sum Z_int,c = 103.40292808166573
eps_c = 8.502626549215508e-16
E_int,c = 474213.87215412565
cV,int = 178.0624305823157
cV,tr = 389.77897414311474
cV = cV,int + cV,tr = 567.8414047254305

Calculating for Carbon, C
Statistical Sum Z_int,c = 2.8117009353772957
eps_c = 1.829647458084777e-17
E_int,c = 44574.30852083491
cV,int = 0.653958872849239
cV,tr = 1038.4197103109216
cV = cV,int + cV,tr = 1039.0736691837708

